In [75]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [76]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup

In [166]:
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [78]:
import warnings
warnings.filterwarnings('ignore')

In [130]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.1493479&myY=126.9218479&searchKeyword='
driver.get(url) #코랩은 화면 안뜸

In [131]:
# 입력창에 광주를 입력하고 검색 버튼을 누르는 동작
driver.find_element(By.ID, 'searchKeyword').send_keys('광주광역시')# 입력창에 광주를 입력
driver.find_element(By.ID, 'fsSearchBtn').click() #검색 버튼을 누르는 동작

### 이동 후에는 BeautifulSoup 이용하여 정보를 획득

In [132]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [133]:
li = lis[0]
name = li.select_one('.title').get_text().strip()
name

'광산소방서_광주-평동-119 안전센터'

In [134]:
addr = li.find('address').string.strip()
addr

'광주광역시 광산구 월전동 1251'

In [135]:
tel = li.select_one('.tel').get_text().strip()
tel

'전화공사중'

In [136]:
lines = []
for li in lis:
  name = li.select_one('.title').get_text().strip()
  addr = li.find('address').string.strip()
  tel = li.select_one('.tel').get_text().strip()
  lines.append([name, addr, tel])

# 두번째 페이지 이동

In [137]:
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()

In [138]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

9

# 다음 페이지 이동

In [139]:
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

<h1><b>전체

In [167]:
# 검색결과 건수 획득
strongs=soup.select('.stations-local-after > p > strong')
strongs[1].get_text()

'29건'

In [168]:
count = int(strongs[1].get_text().replace('건','').strip())
count

29

In [169]:
import math
count = math.ceil(count/10) #ceil 몫+1(딱 떨어지지 않는 경우에만)
count

3

In [170]:
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('광주광역시')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [172]:
lines = []
for i in range(count):
    if i != 0 and i % 2 == 0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
    elif i != 0 and i % 2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').string.strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name, addr, tel])

In [173]:
len(lines)

29

In [174]:
driver.close()

In [175]:
df = pd.DataFrame(lines, columns=['관서명','주소','전화번호'])
df

,관서명,주소,전화번호
0,광산소방서_광주-평동-119 안전센터,광주광역시 광산구 월전동 1251,전화공사중
1,광산소방서_광주-첨단-119 안전센터,광주광역시 광산구 임방울대로825번길 16(쌍암동),062-606-4883
2,광산소방서_광주-신가-119 안전센터,광주광역시 광산구 수등로 236(신가동),062-606-4863
3,광산소방서_광주-비아-119 안전센터,광주광역시 광산구 비아로12번길 10(비아동),062-606-4843
4,광산소방서_광주-월곡-119 안전센터,광주광역시 광산구 사암로 276(월곡동),062-606-4823
5,광산소방서_광주-송정-119 안전센터,광주광역시 광산구 상무대로 93-1(도산동),062-606-4803
6,광산소방서_광주-하남-119 안전센터,광주광역시 광산구 하남산단1번로 13(하남동),062-606-4903
7,북부소방서_광주-동림-119 안전센터,광주광역시 북구 북문대로 202(동림동),062-606-4783
8,북부소방서_광주-두암-119 안전센터,광주광역시 북구 삼정로 5(두암동),062-606-4763
9,북부소방서_광주-일곡-119 안전센터,광주광역시 북구 설죽로 529(일곡동),062-606-4743
